In [ ]:
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import svm

In [ ]:
df = pd.read_csv("1_Assignment_Online Retail.csv")
df=df.sort_values("CustomerID")
df=df.replace(r'^s*$', float('NaN'), regex = True)
df=df.dropna()
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Churn
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,Yes
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,Yes
286628,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-02-08 08:48:00,2.25,12347.0,Iceland,Yes
72263,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes
72264,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes


In [ ]:
#target variable creation
date_1=pd.Timestamp(2010,12,1)
date_2=pd.Timestamp(2011,8,30)
date_3=pd.Timestamp(2011,9,1)
date_4=pd.Timestamp(2011,12,31)

dt1=df[df["InvoiceDate"]>=date_1]
dt1=dt1[dt1["InvoiceDate"]<=date_2]


dt2=df[df["InvoiceDate"]>=date_3]
dt2=dt1[dt1["InvoiceDate"]<=date_4]


In [ ]:
dt1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Churn
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,Yes
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,Yes
286628,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-02-08 08:48:00,2.25,12347.0,Iceland,Yes
72263,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes
72264,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes


In [ ]:
dt2.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Churn
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,Yes
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,Yes
286628,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-02-08 08:48:00,2.25,12347.0,Iceland,Yes
72263,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes
72264,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes


In [ ]:
dt1_id=dt1["CustomerID"].unique()
dt2_id=dt2["CustomerID"].unique()
list_churn=[]
#list of customers who churn
for i in dt1_id:
    if i  in dt2_id:
        list_churn.append(i)

df=df.assign(IsChurn=[0 if x in list_churn else 1 for x in df["CustomerID"]])


In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Churn,IsChurn
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,Yes,0
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,Yes,0
286628,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-02-08 08:48:00,2.25,12347.0,Iceland,Yes,0
72263,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes,0
72264,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,Yes,0


In [ ]:
df=df.drop(['InvoiceDate','InvoiceNo','Description'],axis=1)

df['StockCode'] = df['StockCode'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)


In [ ]:
df.head()

,StockCode,Quantity,UnitPrice,CustomerID,Country,Churn,IsChurn
61619,23166.0,74215,1.04,12346.0,United Kingdom,Yes,0
61624,23166.0,-74215,1.04,12346.0,United Kingdom,Yes,0
286628,21578.0,6,2.25,12347.0,Iceland,Yes,0
72263,47559.0,10,1.25,12347.0,Iceland,Yes,0
72264,21154.0,10,1.25,12347.0,Iceland,Yes,0


In [ ]:
pure_df = df.dropna(axis=1)
ohc_pure_df = pd.get_dummies(pure_df)
ohc_pure_without=ohc_pure_df.drop('IsChurn', axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ohc_pure_without,ohc_pure_df['IsChurn'], test_size=0.15)
model_dt = DecisionTreeClassifier()
model_dt = model_dt.fit(x_train,y_train)
y_pred = model_dt.predict(x_test)


In [ ]:
print("Result Accuracy :",metrics.accuracy_score(y_test, y_pred))

Result Accuracy : 0.9986890618598935


In [ ]:
classifier = svm.SVC(kernel='linear', random_state=0)

x_train, x_test, y_train, y_test = train_test_split(ohc_pure_without,ohc_pure_df['IsChurn'], test_size=0.15)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

ConfusionMatrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test,y_pred)

Clabel = ["Negative", "Positive"]
df = pd.DataFrame(ConfusionMatrix, index = Clabel, columns = Clabel)

print ("Result accuracy : ",accuracy_score(y_test, y_pred)*100)
print(classification_report(y_test, y_pred)) 


plt.title("Test_Confusion_Matrix")
plt.xlabel("Predicted_Label")
plt.ylabel("Actual_Label")
plt.show()

In [ ]:
ANN_model = tf.keras.Sequential()
ANN_model.add(tf.keras.layers.Dense(10, activation='relu'))
ANN_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

ANN_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = ANN_model.fit(x_train, y_train,  epochs=5,batch_size=24, validation_data=(x_test, y_test))
results = ANN_model.evaluate(X_test, y_test, batch_size=24)
print("result accuracy:", results[1])